<a href="https://colab.research.google.com/github/HuixianZhang/Vision-Transformer-Covid/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vit-keras

In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('/content/drive/MyDrive/model_vit_covid.hdf5')
# from vit_keras import vit
# from keras.models import load_model
# import tensorflow as tf
# import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from vit_keras import vit

# warnings.filterwarnings('ignore')
# print('TensorFlow Version ' + tf.__version__)

# vit_model = vit.vit_b32(
#         image_size = 256,
#         activation = 'softmax',
#         pretrained = True,
#         include_top = False,
#         pretrained_top = False,
#         classes = 4)


# model = tf.keras.Sequential([
#         vit_model,
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(4, 'softmax')
#     ],
#     name = 'vision_transformer')
# model = model.load_weights('/content/drive/MyDrive/model_vit_covid_test.hdf5')
# print(model)

In [ ]:

IMAGE_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 20


TEST_PATH = '/content/drive/MyDrive/test_new'


TEST_IMAGES = glob.glob(TEST_PATH + '/*.png')
DF_TEST = pd.DataFrame(TEST_IMAGES, columns = ['image_path'])

classes = {0 : "COVID",
           1 : "Lung_Opacity",
           2 : "Normal",
           3 : "Viral Pneumonia"}

def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower = .7, upper = 1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower = .8, upper = 1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta = .1)
        
    return image


datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          samplewise_center = True,
                                                          samplewise_std_normalization = True,
                                                          validation_split = 0.2,
                                                          preprocessing_function = data_augment)

# train_gen = datagen.flow_from_dataframe(dataframe = DF_TRAIN,
#                                         directory = TRAIN_PATH,
#                                         x_col = 'image_id',
#                                         y_col = 'label',
#                                         subset = 'training',
#                                         batch_size = BATCH_SIZE,
#                                         seed = 1,
#                                         color_mode = 'rgb',
#                                         shuffle = True,
#                                         class_mode = 'categorical',
#                                         target_size = (IMAGE_SIZE, IMAGE_SIZE))

# valid_gen = datagen.flow_from_dataframe(dataframe = DF_TRAIN,
#                                         directory = TRAIN_PATH,
#                                         x_col = 'image_id',
#                                         y_col = 'label',
#                                         subset = 'validation',
#                                         batch_size = BATCH_SIZE,
#                                         seed = 1,
#                                         color_mode = 'rgb',
#                                         shuffle = False,
#                                         class_mode = 'categorical',
#                                         target_size = (IMAGE_SIZE, IMAGE_SIZE))

test_gen = datagen.flow_from_dataframe(dataframe = DF_TEST,
                                       x_col = 'image_path',
                                       y_col = None,
                                       batch_size = BATCH_SIZE,
                                       seed = 1,
                                       color_mode = 'rgb',
                                       shuffle = False,
                                       class_mode = None,
                                       target_size = (IMAGE_SIZE, IMAGE_SIZE))
# images = [train_gen[0][0][i] for i in range(16)]
# fig, axes = plt.subplots(3, 5, figsize = (10, 10))

# axes = axes.flatten()

# for img, ax in zip(images, axes):
#     ax.imshow(img.reshape(IMAGE_SIZE, IMAGE_SIZE, 3))
#     ax.axis('off')

# plt.tight_layout()
# plt.show()

# !pip install --quiet vit-keras



vit_model = vit.vit_b32(
        image_size = IMAGE_SIZE,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 4)


model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4, 'softmax')
    ],
    name = 'vision_transformer')

# model.summary()
# learning_rate = 1e-4

# optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

# model.compile(optimizer = optimizer, 
#               loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2), 
#               metrics = ['accuracy'])

# STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
# STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
#                                                  factor = 0.2,
#                                                  patience = 2,
#                                                  verbose = 1,
#                                                  min_delta = 1e-4,
#                                                  min_lr = 1e-6,
#                                                  mode = 'max')

# earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
#                                                  min_delta = 1e-4,
#                                                  patience = 5,
#                                                  mode = 'max',
#                                                  restore_best_weights = True,
#                                                  verbose = 1)

# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = '/content/drive/MyDrive/model_vit_covid.hdf5',
#                                                   monitor = 'val_accuracy', 
#                                                   verbose = 1, 
#                                                   save_best_only = True,
#                                                   save_weights_only = True,
#                                                   mode = 'max')

# callbacks = [earlystopping, reduce_lr, checkpointer]

# model.fit(x = train_gen,
#           steps_per_epoch = STEP_SIZE_TRAIN,
#           validation_data = valid_gen,
#           validation_steps = STEP_SIZE_VALID,
#           epochs = EPOCHS,
#           callbacks = callbacks)

# # model.save('model.h5', save_weights_only = True)
# model.save('/content/drive/MyDrive/model_vit_covid.h5')
# predicted_classes = np.argmax(model.predict(valid_gen, steps = valid_gen.n // valid_gen.batch_size + 1), axis = 1)
# true_classes = valid_gen.classes
# class_labels = list(valid_gen.class_indices.keys())  

# confusionmatrix = confusion_matrix(true_classes, predicted_classes)
# plt.figure(figsize = (16, 16))
# sns.heatmap(confusionmatrix, cmap = 'Blues', annot = True, cbar = True)

# print(classification_report(true_classes, predicted_classes))
print(model)

Found 4233 validated image filenames.
353263616/353253686 [==============================] - 5s 0us/step


/usr/local/lib/python3.7/dist-packages/vit_keras/utils.py:83: UserWarning: Resizing position embeddings from 12, 12 to 8, 8
  UserWarning,


In [ ]:
model.load_weights('/content/drive/MyDrive/model_vit_covid_test.hdf5')
# print(model.predict(test_gen))

In [ ]:
# print(test_gen.filenames)

In [ ]:
# print(len(test_gen))

In [ ]:
# print(len(test_gen.filenames))

In [ ]:
# for i in range(len(test_gen.filenames)):
#   print(test_gen.filenames[i])

In [ ]:
predicted_classes = np.argmax(model.predict(test_gen), axis = 1)


In [ ]:
# print(predicted_classes)

In [ ]:
true_classes =[]
for i in range(len(test_gen.filenames)):
  # print(test_gen.filenames[i])
  if 'COVID' in test_gen.filenames[i]:
    true_classes.append(0)
  if 'Lung' in test_gen.filenames[i]:
    true_classes.append(1)
  if 'Normal' in test_gen.filenames[i]:
    true_classes.append(2)
  if 'Viral' in test_gen.filenames[i]:
    true_classes.append(3)


print(len(true_classes))

4233


In [ ]:
count =0 
for j in range(len(true_classes)):
  if true_classes[j] == predicted_classes[j]:
    count+=1
acc = count/4233
print(acc)

0.8977084809827546
